In [1]:
import pandas as pd
import numpy as np
import re # For regular expressions (used for title parsing)
from nltk.corpus import stopwords # For tag cleaning
import string # For punctuation

# Define path to the data
DATA_PATH = '../data/' 

In [2]:
# --- Loading the Datasets ---
movies_df = pd.read_csv(DATA_PATH + 'movies.csv')
ratings_df = pd.read_csv(DATA_PATH + 'ratings.csv')
tags_df = pd.read_csv(DATA_PATH + 'tags.csv')
links_df = pd.read_csv(DATA_PATH + 'links.csv')

In [3]:
processed_movies_df = movies_df.copy()

# 1. Extract Year from Title
def extract_year(title):
    year_match = re.search(r'\((\d{4})\)$', title)
    if year_match:
        return int(year_match.group(1))
    return None

processed_movies_df['year'] = processed_movies_df['title'].apply(extract_year)

# Remove year from title string
def clean_title(title):
    return re.sub(r'\s*\(\d{4}\)$', '', title).strip()

processed_movies_df['title_clean'] = processed_movies_df['title'].apply(clean_title)


# 2. Process Genres
# Split pipe-separated genres into a list
processed_movies_df['genres_list'] = processed_movies_df['genres'].apply(lambda x: x.split('|'))

# Handle '(no genres listed)' - replace with an empty list
processed_movies_df['genres_list'] = processed_movies_df['genres_list'].apply(
    lambda genres: [] if genres == ['(no genres listed)'] else genres
)

# Create a space-separated string of genres for TF-IDF
processed_movies_df['genres_str'] = processed_movies_df['genres_list'].apply(lambda x: ' '.join(x))


print("\nProcessed movies_df:")
print(processed_movies_df[['movieId', 'title_clean', 'year', 'genres_list', 'genres_str']].head())
print(processed_movies_df.info())

# Checking for movies with no extracted year
print(f"\nMovies with no extracted year: {processed_movies_df['year'].isnull().sum()}")
print(processed_movies_df[processed_movies_df['year'].isnull()]['title'].tolist())


Processed movies_df:
   movieId                  title_clean    year  \
0        1                    Toy Story  1995.0   
1        2                      Jumanji  1995.0   
2        3             Grumpier Old Men  1995.0   
3        4            Waiting to Exhale  1995.0   
4        5  Father of the Bride Part II  1995.0   

                                         genres_list  \
0  [Adventure, Animation, Children, Comedy, Fantasy]   
1                     [Adventure, Children, Fantasy]   
2                                  [Comedy, Romance]   
3                           [Comedy, Drama, Romance]   
4                                           [Comedy]   

                                    genres_str  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy  
<class 'pandas.core.frame.DataFrame'>
Ra

In [4]:
print("\n--- Preprocessing tags.csv ---")
processed_tags_df = tags_df.copy()

# Define stop words for English
stop_words = set(stopwords.words('english'))

def preprocess_tag(tag):
    if pd.isnull(tag):
        return ""
    tag = str(tag).lower()  # Lowercase
    tag = tag.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    # Tokenize and remove stopwords (simple whitespace tokenization here)
    tokens = tag.split()
    tokens = [word for word in tokens if word not in stop_words and word.isalpha() and len(word) > 1] # Keep only alpha words > 1 char
    return " ".join(tokens)

processed_tags_df['tag_clean'] = processed_tags_df['tag'].apply(preprocess_tag)

# Aggregate cleaned tags by movieId
# Group by movieId and join the cleaned tags into a single string for each movie
movie_tags_agg = processed_tags_df.groupby('movieId')['tag_clean'].apply(
    lambda tags: ' '.join(tag for tag in tags if tag) # Join non-empty tags
).reset_index()

# Rename column for clarity
movie_tags_agg.rename(columns={'tag_clean': 'tags_aggregated_str'}, inplace=True)

print("\nAggregated and cleaned tags per movie:")
print(movie_tags_agg.head())
print(f"Number of movies with aggregated tags: {len(movie_tags_agg)}")


--- Preprocessing tags.csv ---

Aggregated and cleaned tags per movie:
   movieId                           tags_aggregated_str
0        1                               pixar pixar fun
1        2  fantasy magic board game robin williams game
2        3                                     moldy old
3        5                              pregnancy remake
4        7                                        remake
Number of movies with aggregated tags: 1572


In [5]:
print("\n--- Merging DataFrames ---")

# Merge processed movies with aggregated tags
# Use a left merge to keep all movies, even if they don't have tags
movies_master_df = pd.merge(processed_movies_df, movie_tags_agg, on='movieId', how='left')

# Fill NaN for movies that had no tags with an empty string
movies_master_df['tags_aggregated_str'] = movies_master_df['tags_aggregated_str'].fillna('')

# Merge with links_df to get tmdbId (useful for UI later)
movies_master_df = pd.merge(movies_master_df, links_df[['movieId', 'tmdbId']], on='movieId', how='left')


print("\nCombined movies_master_df:")
# Select relevant columns for display
display_cols = ['movieId', 'title_clean', 'year', 'genres_str', 'tags_aggregated_str', 'tmdbId']
print(movies_master_df[display_cols].head())
print(movies_master_df.info())

# Check for any NaNs in crucial text fields after merge
print(f"\nNaNs in 'genres_str': {movies_master_df['genres_str'].isnull().sum()}")
print(f"NaNs in 'tags_aggregated_str': {movies_master_df['tags_aggregated_str'].isnull().sum()}")


--- Merging DataFrames ---

Combined movies_master_df:
   movieId                  title_clean    year  \
0        1                    Toy Story  1995.0   
1        2                      Jumanji  1995.0   
2        3             Grumpier Old Men  1995.0   
3        4            Waiting to Exhale  1995.0   
4        5  Father of the Bride Part II  1995.0   

                                    genres_str  \
0  Adventure Animation Children Comedy Fantasy   
1                   Adventure Children Fantasy   
2                               Comedy Romance   
3                         Comedy Drama Romance   
4                                       Comedy   

                            tags_aggregated_str   tmdbId  
0                               pixar pixar fun    862.0  
1  fantasy magic board game robin williams game   8844.0  
2                                     moldy old  15602.0  
3                                                31357.0  
4                              pregnancy 

In [6]:
print("\n--- Final Checks & Consistency ---")

# Check for NaNs in key columns of movies_master_df
print(f"NaNs in 'title_clean': {movies_master_df['title_clean'].isnull().sum()}")

# Check if all movieIds in ratings exist in our master movie list
movies_in_ratings = ratings_df['movieId'].unique()
movies_in_master = movies_master_df['movieId'].unique()

missing_from_master = set(movies_in_ratings) - set(movies_in_master)
if missing_from_master:
    print(f"Warning: {len(missing_from_master)} movieIds from ratings_df are not in movies_master_df. Example: {list(missing_from_master)[:5]}")
else:
    print("All movieIds from ratings_df are present in movies_master_df.")


--- Final Checks & Consistency ---
NaNs in 'title_clean': 0
All movieIds from ratings_df are present in movies_master_df.


In [7]:
movies_master_df.to_csv(DATA_PATH + 'movies_processed.csv', index=False)
print("\nProcessed 'movies_master_df' saved to 'movies_processed.csv'")


Processed 'movies_master_df' saved to 'movies_processed.csv'
